In [1]:
import numpy as np
from collections import deque
import itertools
import warnings

In [2]:
class ComputationalGraph():
    """Represents a computational graph."""

    def __init__(self):
        # The graph in the form of an adjacency list. 
        # Each key in the dictionary is a Node ID and the corresponding value 
        # is the list of ID's of edges that originate from that Node.
        self.adj = {}
        
        # The set of ID's of edges that belong to the computational graph.
        self.edges = set()
    
    
    def add_node(self, nodeId):
        """Add a node to the graph."""
        try:
            _ = Node.get(nodeId)
        except KeyError:
            raise
       
        if nodeId in self.adj:
            warnings.warn("Warning..... Atempting to add node that already exists in Computational Graph. Ignored.")
            return
        
        self.adj[nodeId] = []

    
    def add_edge(self, edgeId):
        """Add an edge to the graph."""
        try:
            edge = Edge.get(edgeId)
        except KeyError:
            raise

        if edgeId in self.edges:
            warnings.warn("Warning..... Atempting to add edge that already exists in Computational Graph. Ignored.")
            return
        
        if edge.nodeFromId not in self.adj or edge.nodeToId not in self.adj:
            raise ValueError('Both nodes of the edge must belong to the Computational Graph')

        self.edges.add(edgeId)
        self.adj[edge.nodeFromId].append(edgeId)

    
    def getTopSort(self):
        """Return the list of ID's of Nodes in the graph sorted in Topological order.
        This is useful for backward propagation of gradients as the order in which 
        nodes need to be visited while passing gradients backwards is the reverse
        topological ordering.
        """
        
        sortedNodes = []
        
        # We store the in-degrees of the nodes in the graph as part of 
        # Kahn's algorithm for calculating the topological sort.
        inDegree = {} 
        
        # A dictionary to maintain whether we have finished placing a node
        # in it's topologically sorted position.
        done = {}

        # Calculate the in-degrees of all nodes in the graph.
        for nodeId in self.adj:
            done[nodeId] = False
            if nodeId not in inDegree:
                inDegree[nodeId] = 0
            for edgeId in self.adj[nodeId]:
                edge = Edge.get(edgeId)
                if edge.nodeToId not in inDegree:
                    inDegree[edge.nodeToId] = 0
                inDegree[edge.nodeToId] += 1
        
        # Initialize the queue with nodes having in-degree = 0.
        # We always maintain the queue with nodes having in-degree = 0.
        queue = deque()
        for nodeId in inDegree:
            if inDegree[nodeId] == 0:
                queue.append(nodeId)
        
        try:
            # While the queue is not empty, remove the front-most node
            # and place it in the sorted list of nodes.
            while (queue):
                frontNodeId = queue.popleft()
                sortedNodes.append(frontNodeId)
                done[frontNodeId] = True

                # Update the in-degrees of neighbours of the removed node.
                # If any neighbour now has in-degree = 0, then it is pushed 
                # to the queue again.
                for edgeId in self.adj[frontNodeId]:
                    edge = Edge.get(edgeId)
                    # If we encounter a node that we have marked as completed,
                    # it means that we have found a cycle in the graph and 
                    # it is not possible to generate a topological sorting.
                    if done[edge.nodeToId]:
                        raise CycleFoundError('Cycle found: Cannot find topological sorting in a cyclic graph')
                    inDegree[edge.nodeToId] -= 1
                    if inDegree[edge.nodeToId] == 0:
                        queue.append(edge.nodeToId)
        
        except CycleFoundError:
            return []

        return sortedNodes


class CycleFoundError(Exception):
    '''Raised when a cycle is found in what is expected to be an acyclic graph.'''
    pass


class Node():
    """A node in the computational graph. Corresponds to a single Tensor instance."""

    # An iterator that generates ID's for successive
    # instances of the class.
    id_iter = itertools.count()

    # A dictionary that maintains a mapping between instance ID's 
    # and the instances.
    items = {}

    def __init__(self, graph, fromNodesAndGrads=[]):
        """
        
        Args:
            
            graph: The computational graph to which the node belongs.
            
            fromNodesAndGrads: A list of (Node@, grad) tuples such that there is
                               an edge between Node and this instance with grad being
                               the gradient of the tensor at this instance wrt to the tensor 
                               at Node@
        """

        if not isinstance(graph, ComputationalGraph):
            raise ValueError('Node cannot exist independent of a computational graph')

        # Assign an ID and add to the dictionary of all Nodes.
        self.id = next(self.id_iter)
        self.__class__.items[self.id] = self
        
        self.graph = graph
        self.graph.add_node(self.id)

        # Create an Edge instance for every edge and that culminates at
        # the current node and add the Edge to the Computational graph.
        for (fromNodeId, grad) in fromNodesAndGrads:
            try:
                edge = Edge(fromNodeId, self.id, grad)
            except KeyError:
                raise
            self.graph.add_edge(edge.id)

    def __repr__(self):
        return f'Node({self.id})'

    @classmethod
    def get(cls, id):
        """Return a reference to a node instance from its ID."""
        if id in cls.items:
            return cls.items[id]
        else:
            raise KeyError(f'Node with id {id} does not exist')



class Edge():
    """An edge in the computational graph."""

    # An iterator that generates ID's for successive
    # instances of the class.
    id_iter = itertools.count()

    # A dictionary that maintains a mapping between instance ID's 
    # and the instances.
    items = {}

    def __init__(self, nodeFromId, nodeToId, grad):
        """
        
        Args:
            
            nodeFromId: ID of the node from which the edge originates.
            
            nodeToId: ID of the node at which the edge terminates.

            grad: The gradient of the tensor at the terminal node wrt
                  the tensor at the start node.
        """

        try:
            _, _ = Node.get(nodeFromId), Node.get(nodeToId)
        except KeyError:
            raise
        
        # Assign an ID and add to the dictionary of all Nodes.
        self.id = next(self.id_iter)
        self.__class__.items[self.id] = self
        
        self.nodeFromId = nodeFromId
        self.nodeToId = nodeToId
        self.grad = grad

    def __repr__(self):
        return f'Edge({self.id})'

    @classmethod
    def get(cls, id):
        """Return a reference to an edge instance from its ID."""
        if id in cls.items:
            return cls.items[id]
        else:
            raise KeyError(f'Edge with id {id} does not exist')


In [3]:
# Test case for ComputationalGraph and topological sort.
#
# 0 -> 1
# 0 -> 3
# 0 -> 5
# 1 -> 2
# 1 -> 3
# 2 -> 3
# 2 -> 4
# 3 -> 4
# 5 -> 4

graph = ComputationalGraph()
n0 = Node(graph)
n1 = Node(graph, fromNodesAndGrads=[(n0.id, 1)])
n2 = Node(graph)
n3 = Node(graph, fromNodesAndGrads=[(n0.id, 1), (n1.id, 1), (n2.id, 1)])
n4 = Node(graph, fromNodesAndGrads=[(n2.id, 1), (n3.id, 1)])
n5 = Node(graph)
graph.add_edge(Edge(n0.id, n5.id, 1).id)
graph.add_edge(Edge(n5.id, n4.id, 1).id)
graph.getTopSort()



[0, 2, 1, 5, 3, 4]